#   Global Earthquake–Tsunami Risk Assessment
# Phase 2


In [29]:
from google.colab import files
uploaded = files.upload()

Saving earthquake_data_tsunami (1).csv to earthquake_data_tsunami (1) (1).csv


In [30]:
import pandas as pd
df = pd.read_csv('earthquake_data_tsunami (1).csv')

In [31]:
print("Dataset shape:", df.shape)
df.head()

Dataset shape: (782, 13)


,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami
0,7.0,8,7,768,117,0.509,17.0,14.000,-9.7963,159.596,2022,11,1
1,6.9,4,4,735,99,2.229,34.0,25.000,-4.9559,100.738,2022,11,0
2,7.0,3,3,755,147,3.125,18.0,579.000,-20.0508,-178.346,2022,11,1
3,7.3,5,5,833,149,1.865,21.0,37.000,-19.2918,-172.129,2022,11,1
4,6.6,0,2,670,131,4.998,27.0,624.464,-25.5948,178.278,2022,11,1


In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

target_column = 'tsunami'
X = df.drop(columns=[target_column])
y = df[target_column]

# Convert categorical columns if any
X = pd.get_dummies(X, drop_first=True)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data for neural network
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [33]:
# --- Decision Tree Model ---

dt_model = DecisionTreeClassifier(random_state=42, max_depth=6)
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_test)

print("🌳 Decision Tree Results:")
print("Accuracy:", accuracy_score(y_test, dt_pred))
print(classification_report(y_test, dt_pred))

🌳 Decision Tree Results:
Accuracy: 0.9490445859872612
              precision    recall  f1-score   support

           0       0.98      0.93      0.96        91
           1       0.91      0.97      0.94        66

    accuracy                           0.95       157
   macro avg       0.95      0.95      0.95       157
weighted avg       0.95      0.95      0.95       157



In [34]:
# --- Neural Network (MLP) Model ---
mlp_model = MLPClassifier(hidden_layer_sizes=(64, 32),
                          max_iter=1000,
                          early_stopping=True,
                          random_state=42)

mlp_model.fit(X_train_scaled, y_train)
mlp_pred = mlp_model.predict(X_test_scaled)

print("🧠 Neural Network (MLP) Results:")
print("Accuracy:", accuracy_score(y_test, mlp_pred))
print(classification_report(y_test, mlp_pred))

🧠 Neural Network (MLP) Results:
Accuracy: 0.8471337579617835
              precision    recall  f1-score   support

           0       0.90      0.82      0.86        91
           1       0.78      0.88      0.83        66

    accuracy                           0.85       157
   macro avg       0.84      0.85      0.85       157
weighted avg       0.85      0.85      0.85       157



In [35]:
from sklearn.metrics import roc_auc_score
dt_auc = roc_auc_score(y_test, dt_pred)
mlp_auc = roc_auc_score(y_test, mlp_pred)
print(f"Decision Tree ROC-AUC: {dt_auc:.3f}")
print(f"Neural Network ROC-AUC: {mlp_auc:.3f}")


Decision Tree ROC-AUC: 0.952
Neural Network ROC-AUC: 0.851


In [36]:
# --- Compare Models ---
dt_acc = accuracy_score(y_test, dt_pred)
mlp_acc = accuracy_score(y_test, mlp_pred)

print(f"\n✅ Model Comparison:")
print(f"Decision Tree Accuracy: {dt_acc:.3f}")
print(f"Neural Network Accuracy: {mlp_acc:.3f}")

if mlp_acc > dt_acc:
    print("→ Neural Network performed better.")
else:
    print("→ Decision Tree performed better.")

plt.show()


✅ Model Comparison:
Decision Tree Accuracy: 0.949
Neural Network Accuracy: 0.847
→ Decision Tree performed better.


**Results Summary**

The Decision Tree performed better than the Neural Network, with higher accuracy (0.949 vs. 0.847) and ROC-AUC (0.952 vs. 0.851). It was more effective at identifying tsunami events.

Its strength comes from handling non-linear patterns and needing less tuning. It also offers clear decision rules and feature importance, making it easier to understand and reliable for predicting tsunamis based on earthquake data.